# SCC-DFTB SK generation 

Input: DFTB_DB.db - ASE database with coordinates, energies and forces

Output: Slater-Koster files for each pair of elements stored in ./SKF/


In [1]:
import os 
import glob
import re
from shutil import copyfile, copytree
import itertools
import sys
import warnings
import numpy as np
from ase.data import atomic_numbers, atomic_names, atomic_masses, covalent_radii, vdw_radii

base_dir=os.getcwd()

print('Base dir is:', base_dir) 

warnings.filterwarnings('ignore')
sys.path.append('/home/broqvist/Desktop/parautomatik/pgm/skprogs/opt/lib/python3.9/site-packages/')
#print(sys.path)
import argparse
import numpy as np


import sktools.common as sc
import sktools.skgen as skgen
from sktools.skdef import Skdef
from sktools import PACKAGE_VERSION

cwd = os.getcwd()
!source /home/broqvist/Desktop/parautomatik/pgm/skprogs/opt/bin/skprogs-activate.sh
skdef_all = "/home/broqvist/Desktop/parautomatik/pgm/SK_def/skdef.hsd"
SCRIPTNAME = sc.get_script_name()
if not os.path.isdir("SKF/"):
    os.mkdir("SKF/")

import sktools.sk_util as sk_util

os.chdir("SKF/")
print('You are here',os.getcwd())


Base dir is: /home/broqvist/Desktop/parautomatik/EC
You are here /home/broqvist/Desktop/parautomatik/EC/SKF


In [20]:
import ase.db as db
DFT_DB=db.connect("../DFT_DB.db")
elements=[]
for row in DFT_DB.select('1<id<10'):
    structure=row.toatoms()
    a=structure.get_chemical_symbols()
    for i in a: 
        if i not in elements:
            elements.append(i)

elem=''
for i in elements:
    if elem=='':
        elem=i
    else:
        elem=elem+','+i
print(elem)        

C,H,O


In [13]:
format(elements)

"['C', 'H', 'O']"

In [21]:
    
#set up input
parser, subparsers = sk_util.get_parser_and_subparser_container()
sk_util.setup_parser_main(parser)
onecnt_common = sk_util.get_onecnt_common_parser()
sk_util.setup_parser_atom(subparsers, onecnt_common, sk_util.run_atom)
sk_util.setup_parser_denscomp(subparsers, onecnt_common, sk_util.run_denscomp)
sk_util.setup_parser_wavecomp(subparsers, onecnt_common, sk_util.run_wavecomp)
twocnt_common = sk_util.get_twocnt_common_parser()
sk_util.setup_parser_twocnt(subparsers, twocnt_common, sk_util.run_twocnt)
sk_util.setup_parser_sktable(subparsers, twocnt_common, sk_util.run_sktable)

class Args:
    includedirs=[] 
    configfiles=[skdef_all]
    builddir='_build' 
    onecnt_binary='slateratom' 
    twocnt_binary='sktwocnt'
    loglevel='info' 
    element1=elem
    element2=elem 
    dummyrep=True 
    outdir='.'
    func=sk_util.run_sktable

        
args=Args()

#run 
args.func()



[ipykernel_launcher.py] Subcommand sktable started
[ipykernel_launcher.py] Subcommand sktable started
[ipykernel_launcher.py] Subcommand sktable started
[skgen.sktable] Started for C-C
[skgen.sktable] Started for C-C
[skgen.sktable] Started for C-C
[skgen.sktable] Creating free atom prerequisite for C
[skgen.sktable] Creating free atom prerequisite for C
[skgen.sktable] Creating free atom prerequisite for C
[skgen.atom] Started for C
[skgen.atom] Started for C
[skgen.atom] Started for C
[skgen.atom] Calculating spin polarized atom (_build/c/atom.ullfrr4g/atom0_spin)
[skgen.atom] Calculating spin polarized atom (_build/c/atom.ullfrr4g/atom0_spin)
[skgen.atom] Calculating spin polarized atom (_build/c/atom.ullfrr4g/atom0_spin)
[skgen.atom] Calculating spin averaged atom (_build/c/atom.ullfrr4g/atom0)
[skgen.atom] Calculating spin averaged atom (_build/c/atom.ullfrr4g/atom0)
[skgen.atom] Calculating spin averaged atom (_build/c/atom.ullfrr4g/atom0)
[skgen.atom] Calculating Hubbard U value